## Quantitative Momentum Strategy

### Low Quality Momentum Strategy

#### Import Libraries and Data

In [2]:
!pip install xlsxwriter


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.5 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
from statistics import mean

In [66]:
from google.colab import files
uploaded = files.upload()

for filename in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=filename, length=len(uploaded[filename])))


Saving sp_500_stocks.csv to sp_500_stocks.csv
User uploaded file "sp_500_stocks.csv" with length 17599 bytes


In [67]:
stocks = pd.read_csv('sp_500_stocks.csv', encoding='latin1')
stocks.rename(columns={'Symbol': 'Ticker'}, inplace = True)
stocks.head()

,Ticker,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care


#### Batch API Calls

In [68]:
from mysecrets import IEX_CLOUD_API_TOKEN

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

symbol_strings

['MMM,AOS,ABT,ABBV,ABMD,ACN,ATVI,ADM,ADBE,AAP,AMD,AES,AFL,A,APD,AKAM,ALK,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,AME,AMGN,APH,ADI,ANSS,AON,APA,AAPL,AMAT,APTV,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,BKR,BLL,BAC,BBWI,BAX,BDX,BRK.B,BBY,BIO,TECH,BIIB,BLK,BK,BA,BKNG,BWA,BXP,BSX,BMY,AVGO,BR,BRO,BF.B,CHRW,CDNS,CZR,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,CNC,CNP,CERN,CF,CRL,SCHW',
 'CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CTXS,CLX,CME,CMS,KO,CTSH,CL,CMCSA,CMA,CAG,COP,ED,STZ,CPRT,GLW,CTVA,COST,CTRA,CCI,CSX,CMI,CVS,DHI,DHR,DRI,DVA,DE,DAL,XRAY,DVN,DXCM,FANG,DLR,DFS,DISCK,DISH,DG,DLTR,D,DPZ,DOV,DOW,DTE,DUK,DRE,DD,DXC,EMN,ETN,EBAY,ECL,EIX,EW,EA,LLY,EMR,ENPH,ETR,EOG,EFX,EQIX,EQR,ESS,EL,ETSY,EVRG,ES,EXC,EXPE,EXPD,EXR,XOM,FFIV,FB,FAST,FRT,FDX,FIS,FITB,FRC,FE,FISV,FLT,FMC,F,FTNT,FTV,FOXA,FOX',
 'BEN,FCX,GPS,GRMN,IT,GNRC,GD,GE,GIS,GM,GPC,GILD,GPN,GL,GS,HAL,HBI,HAS,HCA,PEAK,HSIC,HES,HPE,HLT,HOLX,HD,HON,HRL,HST,HWM,HPQ,HUM,HBAN,HII,IBM,IEX,IDXX,INFO

In [69]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token=pk_656c6fc4d67549838fd2594899edd797'
    data = requests.get(batch_api_call_url).json()
    # for symbol in symbol_string.split(','):
    #     final_dataframe = final_dataframe.append(
    #                                     pd.Series([symbol,
    #                                                data[symbol]['quote']['latestPrice'],
    #                                                data[symbol]['stats']['year1ChangePercent'],
    #                                                'N/A'
    #                                                ],
    #                                               index = my_columns),
    #                                     ignore_index = True)
    new_rows = []
    for symbol in symbol_string.split(','):
        new_row = pd.Series([symbol,
                             data[symbol]['quote']['latestPrice'],
                             data[symbol]['stats']['year1ChangePercent'],
                             'N/A'
                             ],
                            index=my_columns)
        new_rows.append(new_row)

    # 将新行列表转换为DataFrame并与现有DataFrame合并
    final_dataframe = pd.concat([final_dataframe, pd.DataFrame(new_rows)], ignore_index=True)

final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,MMM,91.845,-0.081477,N/A
1,AOS,86.620,0.311083,N/A
2,ABT,110.240,0.070817,N/A
3,ABBV,162.740,0.045035,N/A
4,ABMD,381.020,0.000000,N/A
...,...,...,...,...
491,YUM,138.860,0.041288,N/A
492,ZBRA,288.870,-0.044213,N/A
493,ZBH,126.810,-0.052335,N/A
494,ZION,40.730,0.429726,N/A


#### Remove Low-Momentum Stocks

In [70]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,NVDA,890.070,2.296900,N/A
1,GPS,22.305,1.518807,N/A
2,AVGO,1354.090,1.243893,N/A
3,NRG,75.030,1.180466,N/A
4,RCL,129.030,1.060856,N/A
5,GE,155.350,1.042938,N/A
6,WDC,72.300,1.032008,N/A
7,LLY,763.030,1.022609,N/A
8,ETN,318.295,1.000383,N/A
9,MU,124.160,0.977363,N/A


#### Calculate Number of Shares to Buy

In [71]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:1000000
1000000


In [72]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,NVDA,890.070,2.296900,22
1,GPS,22.305,1.518807,879
2,AVGO,1354.090,1.243893,14
3,NRG,75.030,1.180466,261
4,RCL,129.030,1.060856,151
5,GE,155.350,1.042938,126
6,WDC,72.300,1.032008,271
7,LLY,763.030,1.022609,25
8,ETN,318.295,1.000383,61
9,MU,124.160,0.977363,157


### High Quality Momentum Strategy

#### Build DataFrame

In [75]:
hqm_columns = [
                'Ticker',
                'Price',
                'Number of Shares to Buy',
                'One-Year Price Return',
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token=pk_656c6fc4d67549838fd2594899edd797'
    data = requests.get(batch_api_call_url).json()
    # for symbol in symbol_string.split(','):
    #     hqm_dataframe = hqm_dataframe.append(
    #                                     pd.Series([symbol,
    #                                                data[symbol]['quote']['latestPrice'],
    #                                                'N/A',
    #                                                data[symbol]['stats']['year1ChangePercent'],
    #                                                'N/A',
    #                                                data[symbol]['stats']['month6ChangePercent'],
    #                                                'N/A',
    #                                                data[symbol]['stats']['month3ChangePercent'],
    #                                                'N/A',
    #                                                data[symbol]['stats']['month1ChangePercent'],
    #                                                'N/A',
    #                                                'N/A'
    #                                                ],
    #                                               index = hqm_columns),
    #                                     ignore_index = True)
    new_rows = []
    for symbol in symbol_string.split(','):
        if symbol in data:
            new_row = pd.Series([
                symbol,
                data[symbol]['quote'].get('latestPrice', 'N/A'),
                'N/A',
                data[symbol]['stats'].get('year1ChangePercent', 'N/A'),
                'N/A',
                data[symbol]['stats'].get('month6ChangePercent', 'N/A'),
                'N/A',
                data[symbol]['stats'].get('month3ChangePercent', 'N/A'),
                'N/A',
                data[symbol]['stats'].get('month1ChangePercent', 'N/A'),
                'N/A',
                'N/A'
                ],
                index = hqm_columns)
            new_rows.append(new_row)
        else:
            print(f"Data for {symbol} not found or incomplete in API response.")

    # 将新行列表转换为DataFrame并与现有DataFrame合并
    hqm_dataframe = pd.concat([hqm_dataframe, pd.DataFrame(new_rows)], ignore_index=True)

hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

#### Calculate Momentum Percentiles

In [76]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
# for time_period in time_periods:
#     print(hqm_dataframe[f'{time_period} Return Percentile'])

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,91.795,N/A,-0.081477,0.201613,0.063231,0.3125,-0.141448,0.0625,-0.114527,0.046371,N/A
1,AOS,86.660,N/A,0.311083,0.743952,0.241845,0.701613,0.070649,0.633065,-0.001385,0.604839,N/A
2,ABT,110.220,N/A,0.070817,0.445565,0.212511,0.655242,-0.037472,0.227823,-0.077399,0.116935,N/A
3,ABBV,162.950,N/A,0.045035,0.41129,0.117602,0.427419,0.008593,0.417339,-0.096051,0.072581,N/A
4,ABMD,381.020,N/A,0.000000,0.339718,0.000000,0.198589,0.000000,0.371976,0.000000,0.634073,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,138.890,N/A,0.041288,0.409274,0.190492,0.602823,0.072482,0.639113,0.004666,0.6875,N/A
492,ZBRA,289.370,N/A,-0.044213,0.256048,0.399961,0.885081,0.163981,0.864919,0.028073,0.824597,N/A
493,ZBH,126.580,N/A,-0.052335,0.237903,0.198110,0.625,0.013359,0.431452,-0.008080,0.540323,N/A
494,ZION,40.800,N/A,0.429726,0.836694,0.203311,0.643145,-0.024024,0.284274,0.021859,0.790323,N/A


#### Calculate High Quality Momentum (HQM) Score

The HQM Score is the arithmetic mean of the 4 momentum percentile scores.

In [77]:
for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,91.795,N/A,-0.081477,0.201613,0.063231,0.3125,-0.141448,0.0625,-0.114527,0.046371,0.155746
1,AOS,86.660,N/A,0.311083,0.743952,0.241845,0.701613,0.070649,0.633065,-0.001385,0.604839,0.670867
2,ABT,110.220,N/A,0.070817,0.445565,0.212511,0.655242,-0.037472,0.227823,-0.077399,0.116935,0.361391
3,ABBV,162.950,N/A,0.045035,0.41129,0.117602,0.427419,0.008593,0.417339,-0.096051,0.072581,0.332157
4,ABMD,381.020,N/A,0.000000,0.339718,0.000000,0.198589,0.000000,0.371976,0.000000,0.634073,0.386089
...,...,...,...,...,...,...,...,...,...,...,...,...
491,YUM,138.890,N/A,0.041288,0.409274,0.190492,0.602823,0.072482,0.639113,0.004666,0.6875,0.584677
492,ZBRA,289.370,N/A,-0.044213,0.256048,0.399961,0.885081,0.163981,0.864919,0.028073,0.824597,0.707661
493,ZBH,126.580,N/A,-0.052335,0.237903,0.198110,0.625,0.013359,0.431452,-0.008080,0.540323,0.458669
494,ZION,40.800,N/A,0.429726,0.836694,0.203311,0.643145,-0.024024,0.284274,0.021859,0.790323,0.638609


#### Select 50 Best Momentum Stocks

In [78]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:51]
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NRG,74.810,N/A,1.180466,0.993952,0.832809,0.995968,0.468686,0.993952,0.166272,0.989919,0.993448
1,GE,155.495,N/A,1.042938,0.989919,0.779977,0.989919,0.499127,0.997984,0.166847,0.991935,0.99244
2,MU,124.155,N/A,0.977363,0.981855,0.776721,0.987903,0.488510,0.995968,0.341336,1.0,0.991431
3,WDC,72.330,N/A,1.032008,0.987903,0.576696,0.979839,0.440956,0.991935,0.185115,0.995968,0.988911
4,ETN,318.700,N/A,1.000383,0.983871,0.537745,0.971774,0.319637,0.979839,0.060641,0.923387,0.964718
5,AVGO,1354.745,N/A,1.243893,0.995968,0.548986,0.97379,0.223768,0.949597,0.073893,0.9375,0.964214
6,CMG,2970.180,N/A,0.701432,0.953629,0.660193,0.983871,0.301818,0.977823,0.076070,0.941532,0.964214
7,MPC,208.595,N/A,0.632212,0.9375,0.423367,0.905242,0.332326,0.983871,0.104720,0.971774,0.949597
8,AMAT,211.160,N/A,0.868146,0.975806,0.484075,0.953629,0.378943,0.989919,0.035417,0.846774,0.941532
9,PSX,162.270,N/A,0.586197,0.933468,0.493360,0.955645,0.231761,0.953629,0.052533,0.91129,0.938508


#### Calculate Number of Shares to Buy

In [79]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:1000000
1000000


In [80]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,NRG,74.810,262,1.180466,0.993952,0.832809,0.995968,0.468686,0.993952,0.166272,0.989919,0.993448
1,GE,155.495,126,1.042938,0.989919,0.779977,0.989919,0.499127,0.997984,0.166847,0.991935,0.99244
2,MU,124.155,157,0.977363,0.981855,0.776721,0.987903,0.488510,0.995968,0.341336,1.0,0.991431
3,WDC,72.330,271,1.032008,0.987903,0.576696,0.979839,0.440956,0.991935,0.185115,0.995968,0.988911
4,ETN,318.700,61,1.000383,0.983871,0.537745,0.971774,0.319637,0.979839,0.060641,0.923387,0.964718
5,AVGO,1354.745,14,1.243893,0.995968,0.548986,0.97379,0.223768,0.949597,0.073893,0.9375,0.964214
6,CMG,2970.180,6,0.701432,0.953629,0.660193,0.983871,0.301818,0.977823,0.076070,0.941532,0.964214
7,MPC,208.595,93,0.632212,0.9375,0.423367,0.905242,0.332326,0.983871,0.104720,0.971774,0.949597
8,AMAT,211.160,92,0.868146,0.975806,0.484075,0.953629,0.378943,0.989919,0.035417,0.846774,0.941532
9,PSX,162.270,120,0.586197,0.933468,0.493360,0.955645,0.231761,0.953629,0.052533,0.91129,0.938508


### Export High Quality Momentum Strategy Results to Excel

In [81]:
writer = pd.ExcelWriter('high_quality_momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [82]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [84]:
writer.close()